In [1]:
import pandas as pd
from classifier.classifier import Classifier
from classifier.classifier_factory import ClassifierFactory
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
from processor.processor import Processor as p
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np
from modeler_strats.universal_modeler import UniversalModeler

In [2]:
## initializing classifier_class
classifier_class = ClassifierFactory.build(Classifier.WEEKLY_STOCK_SPECULATION_CLASSIFIER)

In [3]:
backtest_start_date = datetime(2016,1,1)
backtest_end_date = datetime(2023,1,1)

In [4]:
## creating database objects
market = Market()
sec = SEC()
modeler_strat = UniversalModeler()

In [5]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [6]:
market.connect()
sec.connect()
training_sets = []
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        prices = market.retrieve_ticker_prices(classifier_class.asset_class.value,ticker)
        prices = p.column_date_processing(prices)
        ticker_data = classifier_class.training_set(ticker,prices)
        training_sets.append(ticker_data)
    except Exception as e:
        print(str(e))
        continue
market.disconnect()
sec.disconnect()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:35<00:00, 13.98it/s]


In [7]:
data = pd.concat(training_sets)

In [9]:
training_data = data.dropna().copy().sort_values(["year","week"])

In [10]:
training_data

,d1,d2,d3,rolling14,year,week,ticker,adjclose,y
13,0.041247,12.190637,-13.811506,24.512802,2000,14,MMM,25.077294,True
13,0.042171,0.910210,-2.223177,2.044241,2000,14,AOS,2.011732,True
13,0.088651,3.427573,-7.301858,8.389861,2000,14,ABT,9.418513,True
13,-0.095087,-0.303730,-1.052338,27.135143,2000,14,ABMD,19.338000,False
13,-0.199006,4.427577,-1.470438,1.062031,2000,14,ATVI,0.761955,False
...,...,...,...,...,...,...,...,...,...
1221,-0.001266,-0.975455,-0.929091,132.631163,2023,22,YUM,130.815000,True
1221,0.004525,-1.195196,-0.569230,287.096036,2023,22,ZBRA,268.640000,True
1141,-0.016909,-0.566203,-1.257546,130.548234,2023,22,ZBH,127.735000,True
1221,-0.015722,-1.126086,-3.409437,30.822367,2023,22,ZION,28.235000,True


In [11]:
classifier_class.db.connect()
model_start_date = backtest_start_date - timedelta(classifier_class.time_horizon_class.model_date_offset)
model_end_date = backtest_end_date - timedelta(classifier_class.time_horizon_class.model_date_offset)
for year in range(model_start_date.year,model_end_date.year):
    training_slice = training_data[(training_data["year"]<year) & (training_data["year"] >= year - 4)].reset_index(drop=True)
    prediction_set = training_data[training_data["year"]==year].reset_index(drop=True)
    stuff = modeler_strat.classification_model(training_slice,prediction_set,classifier_class.factors,False)
    stuff = stuff.rename(columns={"prediction":f"classification_prediction"})
    relevant_columns = list(set(list(stuff.columns)) - set(classifier_class.factors))
    stuff = classifier_class.sim_processor(stuff)
    classifier_class.db.store("sim",stuff[relevant_columns])
classifier_class.db.disconnect()

Epoch 1/10
2219/2219 [==============================] - 2s 826us/step - loss: 1487440.3750 - accuracy: 0.4719
Epoch 2/10
2219/2219 [==============================] - 2s 807us/step - loss: 621052864.0000 - accuracy: 0.4405
Epoch 3/10
2219/2219 [==============================] - 2s 798us/step - loss: 75518512.0000 - accuracy: 0.4397
Epoch 4/10
2219/2219 [==============================] - 2s 793us/step - loss: 7840503.5000 - accuracy: 0.4392
Epoch 5/10
2219/2219 [==============================] - 2s 806us/step - loss: 3079752.2500 - accuracy: 0.4390
Epoch 6/10
2219/2219 [==============================] - 2s 811us/step - loss: 3407205.0000 - accuracy: 0.4392
Epoch 7/10
2219/2219 [==============================] - 2s 804us/step - loss: 27291972.0000 - accuracy: 0.4392
Epoch 8/10
2219/2219 [==============================] - 2s 814us/step - loss: 1213988.1250 - accuracy: 0.4389
Epoch 9/10
2219/2219 [==============================] - 2s 821us/step - loss: 1625223.3750 - accuracy: 0.4388
Epoch 

2370/2370 [==============================] - 2s 787us/step - loss: 60630392.0000 - accuracy: 0.4322
Epoch 9/10
2370/2370 [==============================] - 2s 791us/step - loss: 484862880.0000 - accuracy: 0.4327
Epoch 10/10
796/796 [==============================] - 0s 538us/step
